###Race Result Dataframe Creation

0. Import configuration

In [0]:
%run ../Includes/Configuration

1. Imports

In [0]:
gold_container_path

In [0]:
from pyspark.sql.functions import current_timestamp, desc, col, max

2. Read appropriate Datasources
- Datasources
  - Circuits Data
  - Races Data
  - PitStops Data
  - Drivers Data
  - Constructors Data
  - Results Data

In [0]:
# Circuits Data

circuits_df = spark.read.parquet(f'{silver_container_path}/circuits') \
    .select('circuit_id', 'name', 'location') \
    .withColumnRenamed('location', 'circuit_location') \
    .withColumnRenamed('name', 'circuit_name')

In [0]:
# Races Data

races_df = spark.read.parquet(f'{silver_container_path}/races') \
    .select('race_id', 'circuit_id','race_year', 'name', 'race_timestamp') \
    .withColumnRenamed('race_timestamp', 'race_date') \
    .withColumnRenamed('name', 'race_name')

In [0]:
# PitStops Data

pitstop_df = spark.read.parquet(f'{silver_container_path}/pitstops') \
    .select('race_id', 'driver_id', 'stop') \
    .withColumnRenamed('stop', 'pits')

In [0]:
# Drivers Data

drivers_df = spark.read.parquet(f'{silver_container_path}/drivers') \
    .select('driver_id', 'number', 'name', 'nationality') \
    .withColumnRenamed('name', 'driver_name') \
    .withColumnRenamed('nationality', 'driver_nationality') \
    .withColumnRenamed('number', 'driver_number')

In [0]:
# Constructors Data

constructor_df = spark.read.parquet(f'{silver_container_path}/constructors') \
    .select('constructor_id', 'name') \
    .withColumnRenamed('name', 'team')

In [0]:
# Results Data

results_df = spark.read.parquet(f'{silver_container_path}/results') \
    .select('result_id', 'race_id','driver_id', 'constructor_id', 'grid', 'fastest_lap_time', 'time', 'points') \
    .withColumnRenamed('time', 'race_time') \
    .withColumnRenamed('fastest_lap_time', 'fastest_lap')

3. Join the Dataframes to get the desired result

In [0]:
# Join Circuits and Races DataFrame

race_circuits_df = races_df.join(circuits_df, races_df.circuit_id == circuits_df.circuit_id, 'inner') \
    .select('race_id', 'race_year', 'race_name', 'race_date', 'circuit_name','circuit_location')

In [0]:
# Join Drivers, race_circuits and Pits
driver_race_circuits_pits_df = pitstop_df.join(race_circuits_df, race_circuits_df.race_id == pitstop_df.race_id, 'inner') \
    .join(drivers_df, pitstop_df.driver_id == drivers_df.driver_id, 'inner') \
    .groupBy(race_circuits_df.race_id, drivers_df.driver_id, race_circuits_df.race_year, race_circuits_df.race_name, race_circuits_df.race_date, race_circuits_df.circuit_name, race_circuits_df.circuit_location, drivers_df.driver_name, drivers_df.driver_number, drivers_df.driver_nationality) \
    .agg(max('pits').alias('pits'))

In [0]:
# Join Results with race_circuits, drivers and constructors DataFrame

race_results_df = results_df.join(driver_race_circuits_pits_df, [results_df.race_id == driver_race_circuits_pits_df.race_id, results_df.driver_id == driver_race_circuits_pits_df.driver_id], 'inner') \
    .join(constructor_df, results_df.constructor_id == constructor_df.constructor_id, 'inner') \
    .select('race_year', 'race_name', 'race_date', 'circuit_name','circuit_location', 'driver_name', 'driver_number', 'driver_nationality', 'team', 'grid', 'pits','fastest_lap', 'race_time', 'points') \
    .withColumn('created_date', current_timestamp())

4. Write to Gold container

In [0]:
race_results_df.write.mode('overwrite').parquet(f'{gold_container_path}/race_results')